##### Copyright 2023 The IREE Authors

In [1]:
#@title Licensed under the Apache License v2.0 with LLVM Exceptions.
# See https://llvm.org/LICENSE.txt for license information.
# SPDX-License-Identifier: Apache-2.0 WITH LLVM-exception

# Dynamic Shapes

This notebook

1. Creates a PyTorch program with dynamic shapes using [iree-turbine](https://github.com/iree-org/iree-turbine)'s advanced AOT toolkit
2. Compiles the program to an IREE VM bytecode module
3. Tests running the compiled VM module using IREE's runtime
4. Downloads compilation artifacts for use with the native (C API) sample application

In [2]:
#@title General setup

import os
import tempfile

ARTIFACTS_DIR = os.path.join(tempfile.gettempdir(), "iree", "colab_artifacts")
os.makedirs(ARTIFACTS_DIR, exist_ok=True)
print(f"Using artifacts directory '{ARTIFACTS_DIR}'")

Using artifacts directory '/tmp/iree/colab_artifacts'


In [3]:
%%capture
#@title Uninstall existing packages
#   This avoids some warnings when installing specific PyTorch packages below.
!python -m pip uninstall -y fastai torchaudio torchdata torchtext torchvision

In [4]:
#@title Install iree-turbine

# Limit cell height.
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

!python -m pip install iree-turbine

<IPython.core.display.Javascript object>

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.2/71.2 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: ml_dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires ml-dtypes<0.5.0,>=0.3.1, but you have ml-dtypes 0.5.1 which is incompatible.


In [5]:
#@title Report version information
!echo "Installed iree-turbine, $(python -m pip show iree_turbine | grep Version)"

!echo -e "\nInstalled IREE, compiler version information:"
!iree-compile --version

import torch
print("\nInstalled PyTorch, version:", torch.__version__)

Installed iree-turbine, Version: 3.1.0

Installed IREE, compiler version information:
IREE (https://iree.dev):
  IREE compiler version 3.1.0rc20250107 @ d2242207764230ad398585a5771f9d54ce91b4c8
  LLVM version 20.0.0git
  Optimized build

Installed PyTorch, version: 2.5.1+cu121


## Create a program using PyTorch + iree-turbine

NOTE: as in other domains, providing more information to a compiler allows it
to generate more efficient code. As a general rule, the slowest varying
dimensions of program data like batch index or timestep are safer to treat as
dynamic than faster varying dimensions like image x/y/channel. See
[this paper](https://arxiv.org/pdf/2006.03031.pdf) for a discussion of the
challenges imposed by dynamic shapes and one project's approach to addressing
them.

In [6]:
#@title Define a sample `torch.nn.Module`.

import iree.turbine.aot as aot

class DynamicShapesModule(torch.nn.Module):
  # reduce_sum_1d (dynamic input size, static output size)
  #   tensor<?xi32> -> tensor<i32>
  #   e.g. [1, 2, 3] -> 6
  def reduce_sum_1d(self, values):
    return torch.sum(values)

  # reduce_sum_2d (partially dynamic input size, static output size)
  #   tensor<?x3xi32> -> tensor<3xi32>
  #   e.g. [[1, 2, 3], [10, 20, 30]] -> [11, 22, 33]
  def reduce_sum_2d(self, values):
    return torch.sum(values, 0)

  # add_one (dynamic input size, dynamic output size)
  #   tensor<?xi32>) -> tensor<?xi32>
  #   e.g. [1, 2, 3] -> [2, 3, 4]
  def add_one(self, values):
    return values + 1

In [7]:
#@title Export using FxProgramsBuilder.

fxb = aot.FxProgramsBuilder(DynamicShapesModule())

# Create a single dynamic export dimension.
dynamic_x = torch.export.Dim("x")
# Example inputs with a mix of placeholder (dynamic) and static dimensions.
example_1d = torch.empty(16, dtype=torch.int32)
example_2d = torch.empty((16, 3), dtype=torch.int32)

# Export reduce_sum_1d with a dynamic dimension.
@fxb.export_program(
    args=(example_1d,),
    dynamic_shapes={"values": {0: dynamic_x}},
)
def reduce_sum_1d(module, values):
    return module.reduce_sum_1d(values)

# Export reduce_sum_2d with one dynamic dimension.
@fxb.export_program(
    args=(example_2d,),
    dynamic_shapes={"values": {0: dynamic_x}},
)
def reduce_sum_2d(module, values):
    return module.reduce_sum_2d(values)

# Export add_one with a dynamic dimension.
@fxb.export_program(
    args=(example_1d,),
    dynamic_shapes={"values": {0: dynamic_x}},
)
def add_one(module, values):
    return module.add_one(values)

export_output = aot.export(fxb)

In [8]:
from iree.compiler.ir import Context

imported_mlir_path = os.path.join(ARTIFACTS_DIR, "dynamic_shapes.mlir")
export_output.save_mlir(imported_mlir_path)
print(f"Wrote MLIR to path '{imported_mlir_path}'")

# Inspect the IR.
# Note the question marks for dynamic shapes in types, like `tensor<?xi32>`.
print("\nDynamic Shapes MLIR:")
!cat {imported_mlir_path}

Wrote MLIR to path '/tmp/iree/colab_artifacts/dynamic_shapes.mlir'

Dynamic Shapes MLIR:
module @module {
  func.func @reduce_sum_1d(%arg0: !torch.vtensor<[?],si32>) -> !torch.vtensor<[],si64> attributes {torch.assume_strict_symbolic_shapes} {
    %none = torch.constant.none
    %0 = torch.aten.sum %arg0, %none : !torch.vtensor<[?],si32>, !torch.none -> !torch.vtensor<[],si64>
    return %0 : !torch.vtensor<[],si64>
  }
  func.func @reduce_sum_2d(%arg0: !torch.vtensor<[?,3],si32>) -> !torch.vtensor<[3],si64> attributes {torch.assume_strict_symbolic_shapes} {
    %int0 = torch.constant.int 0
    %0 = torch.prim.ListConstruct %int0 : (!torch.int) -> !torch.list<int>
    %false = torch.constant.bool false
    %none = torch.constant.none
    %1 = torch.aten.sum.dim_IntList %arg0, %0, %false, %none : !torch.vtensor<[?,3],si32>, !torch.list<int>, !torch.bool, !torch.none -> !torch.vtensor<[3],si64>
    return %1 : !torch.vtensor<[3],si64>
  }
  func.func @add_one(%arg0: !torch.vtensor<[?],si

## Test the imported program

_Note: you can stop after each step and use intermediate outputs with other tools outside of Colab._

_See the [README](https://github.com/iree-org/iree/tree/main/samples/dynamic_shapes#instructions) for more details and example command line instructions._

* _The "imported MLIR" (above) can be used by IREE's generic compiler tools_
* _The "binary" can be saved and used by runtime applications_

_The specific point at which you switch from Python to native tools will depend on your project._

In [9]:
# Compile to a file on disk for usage outside of Python.
flatbuffer_path = os.path.join(ARTIFACTS_DIR, "dynamic_shapes_cpu.vmfb")
export_output.compile(save_to=flatbuffer_path)
print(f"Wrote compiled program to path '{flatbuffer_path}'")

# Compile into memory for testing.
binary = export_output.compile(save_to=None)

Wrote compiled program to path '/tmp/iree/colab_artifacts/dynamic_shapes_cpu.vmfb'


In [10]:
import iree.runtime as ireert
import numpy as np

# Use the IREE runtime API to test the compiled program.
config = ireert.Config("local-task")
vm_module = ireert.load_vm_module(
    ireert.VmModule.wrap_buffer(config.vm_instance, binary.map_memory()),
    config,
)

print(vm_module.reduce_sum_1d(np.array([1, 10, 100], dtype=np.int32)).to_host())
print(vm_module.reduce_sum_2d(np.array([[1, 2, 3], [10, 20, 30]], dtype=np.int32)).to_host())
print(vm_module.reduce_sum_2d(np.array([[1, 2, 3], [10, 20, 30], [100, 200, 300]], dtype=np.int32)).to_host())
print(vm_module.add_one(np.array([1, 10, 100], dtype=np.int32)).to_host())

111
[11 22 33]
[111 222 333]
[  2  11 101]


## Download compilation artifacts

In [11]:
ARTIFACTS_ZIP = "/tmp/dynamic_shapes_colab_artifacts.zip"

print(f"Zipping '{ARTIFACTS_DIR}' to '{ARTIFACTS_ZIP}' for download...")
!cd {ARTIFACTS_DIR} && zip -r {ARTIFACTS_ZIP} .

# Note: you can also download files using Colab's file explorer
try:
  from google.colab import files
  print("Downloading the artifacts zip file...")
  files.download(ARTIFACTS_ZIP)
except ImportError:
  print("Missing google_colab Python package, can't download files")

Zipping '/tmp/iree/colab_artifacts' to '/tmp/dynamic_shapes_colab_artifacts.zip' for download...
  adding: dynamic_shapes_cpu.vmfb (deflated 66%)
  adding: dynamic_shapes.mlir (deflated 72%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>